In [33]:
import os, nltk, collections
import pandas as pd
import collections
from nltk.tokenize import word_tokenize, sent_tokenize
from pprint import pprint
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from ast import literal_eval
data = pd.read_pickle("small_data_pickle")


In [109]:
relevant = ['fake','reliable']
data = data[data["simple_type"].isin(relevant) == True] #fjerne ikke-relevante rækker, simple type tracker definitionen af fake news 
print("row count after removal:", len(data))

X = data.drop(["type"],axis=1).drop(["simple_type"],axis=1) #lav x-vector


row count after removal: 195


In [87]:
#get vocabulary
def vocabulary_size(series):
    '''Computes the vocabulary size of a corpus by iterating through every word
    in every document in the corpus and adding it to a dictionary if it hasnt been seen before
    as well as incrementing a variable counting every unique word. Returns the unique_word_count
    and sorted list of words and their frequencies'''
    # Initialize a counter for unique words count and dictionary to store word frequencies
    unique_words_count = 0
    word_frequency = {}
    # Iterate through each element in the series
    for field in series:
        # Iterate through each word in the current element
        for word in field:
            if word in word_frequency: 
                # If word already in dict, increment its count
                word_frequency[word] += 1
            else:
                # If not add it to the dictionary with count 1 and increment unique_words_count
                word_frequency[word] = 1
                unique_words_count += 1
    # Sort the word_frequency dictionary by frequency in descending order
    return word_frequency

#vocabulary is key in dict^
vocabulary = [*vocabulary_size(data["content_stemmed"])]

print(vocabulary)
print(len(vocabulary))

['power', 'christma', 'make', 'wild', 'wonder', 'thing', 'holi', 'triniti', 'posit', 'good', 'simpl', 'act', 'give', 'receiv', 'lost', 'day', 'worri', 'money', 'success', 'hold', 'back', 'congreg', 'ohio', 'move', 'action', 'a', 'sermon', 'church', 'eve', 'pastor', 'grand', 'lake', 'unit', 'methodist', 'celina', 'gave', 'emot', 'import', 'understand', 'messag', 'jesus', 'religi', 'peopl', 'suffer', 'enjoy', 'life', 'bit', 'generos', 'live', 'long', 'time', 'ago', 'generous', 'fashion', 'focus', 'potenc', 'end', 'decid', 'offer', 'bowl', 'pass', 'room', 'pitch', 'word', 'ring', 'ear', 'member', 'drove', 'local', 'waffl', 'hous', 'visit', 'ladi', 'work', 'night', 'shift', 'great', 'choic', 'famili', 'paid', 'bill', 'understood', 'sacrific', 'made', 'want', 'donat', 'entir', 'split', 'total', 'amount', 'cur0000', 'staff', 'beauti', 'moment', 'perfect', 'preacher', 'talk', 'deed', 'help', 'eas', 'burden', 'felt', 'gift', 'communiti', 'part', 'event', 'return', 'pain', 'group', 'sought', 'a

In [110]:
#getting binary target
def bin_target(x):
    if x=="reliable":
        return 0
    if x=="fake":
        return 1

In [100]:
#making a dataset with BoW representation of stemmed articles
from collections import Counter

f = lambda x: Counter([y for y in x if y in vocabulary])
BoWdata = (pd.DataFrame(data['content_stemmed'].apply(f).values.tolist())
               .fillna(0)
               .astype(int)
               .reindex(columns=vocabulary))

#tæller det som leakage at man bruger vocabulary fra hele datasættet??

In [104]:
#def fun(doc):
 #   return doc

#tfidf = TfidfVectorizer(
  #  analyzer='word',
    #tokenizer=fun,
   # preprocessor=fun,
   # token_pattern=None)  

#tfidf.fit(data["content_stemmed"]) #vær opmærksom på at vocab nu er relativt til træningssættet, men det skal det jo sådan set også være, ellers ville der være leakage
#tfidf.vocabulary_
#X_tfidf = data["content_stemmed"].apply(lambda x: tfidf.transform([x]))
#X_tfidf= data["content_stemmed"].apply(lambda x: tfidf.transform([x]))


In [112]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
#making BoWdata ready
X = BoWdata
y = data["simple_type"].apply(lambda x:bin_target(x)).astype(int)
X_train, X_val, y_train, y_val= train_test_split(X,y,test_size=0.2,random_state=0,shuffle=True)
X_val, X_test, y_val, y_test = train_test_split(X_val,y_val, test_size=0.5,random_state=0,shuffle=True) 


In [102]:
#neural network with BoW
clf = MLPClassifier(random_state=1, max_iter=300).fit(X_train, y_train)
clf.score(X_val,y_val)

0.9473684210526315

In [120]:
#making data ready: tf-idf representation
X = data.drop(["type"],axis=1).drop(["simple_type"],axis=1) #lav x-vector
y = data["simple_type"].apply(lambda x:bin_target(x)).astype(int)

X_train, X_val, y_train, y_val= train_test_split(X,y,test_size=0.2,random_state=0,shuffle=True)
X_val, X_test, y_val, y_test = train_test_split(X_val,y_val, test_size=0.5,random_state=0,shuffle=True) 

print(len(X_val))
print(len(X_train))

vectorizer = TfidfVectorizer(
        sublinear_tf=True, max_df=0.5, min_df=5, stop_words="english"
    )

vectorizer.fit(X_train["content"])
X_train = vectorizer.transform(X_train["content"])
X_test = vectorizer.transform(X_val["content"]) #important that vectorizer is only fittet on training data - otherwise leakage
print(X_test.shape)
print(X_train.shape)

clf = MLPClassifier(random_state=1, max_iter=300).fit(X_train, y_train)
clf.score(X_test,y_val)


19
156
(19, 1754)
(156, 1754)


0.9473684210526315

Alt herunder skal nok ikke bruges

In [121]:
from sklearn.linear_model import RidgeClassifier

clf = RidgeClassifier(tol=1e-2, solver="sparse_cg")
clf.fit(X_train, y_train)
pred = clf.predict(X_test)

In [12]:
#tfIdfTransformer = TfidfTransformer(use_idf=True)
#countVectorizer = CountVectorizer()
#X_BoW = countVectorizer.fit_transform(X_train)
#det virker lidt som om den antager at den ikke får en tokenized tekst?

AttributeError: 'list' object has no attribute 'lower'

In [49]:
def fun(doc):
    return doc

tfidf = TfidfVectorizer(
    analyzer='word',
    tokenizer=fun,
    preprocessor=fun,
    token_pattern=None)  

tfidf.fit(X_train["content_stemmed"]) #vær opmærksom på at vocab nu er relativt til træningssættet, men det skal det jo sådan set også være, ellers ville der være leakage
tfidf.vocabulary_
X_tfidf = X_train["content_stemmed"].apply(lambda x: tfidf.transform([x]))
X_tfidf = X_train["content_stemmed"].apply(lambda x: tfidf.transform([x]))


In [50]:
clf = MLPClassifier(random_state=1, max_iter=300).fit(X_tfidf, y_train)
clf.score(X_val,y_val)


71       (0, 9709)\t0.027082482358835893\n  (0, 9685)...
160      (0, 9638)\t0.02626083399350609\n  (0, 9599)\...
180      (0, 9650)\t0.055103682695849705\n  (0, 9602)...
75       (0, 9607)\t0.03748492461728866\n  (0, 9586)\...
59       (0, 9520)\t0.10499222703737843\n  (0, 9438)\...
                             ...                        
67       (0, 9651)\t0.0112136559676212\n  (0, 9622)\t...
192      (0, 9401)\t0.07658557914355257\n  (0, 9151)\...
117      (0, 9560)\t0.03211953463053134\n  (0, 9504)\...
47       (0, 9674)\t0.022602832963982845\n  (0, 9651)...
172      (0, 8990)\t0.27707159300210976\n  (0, 8673)\...
Name: content_stemmed, Length: 200, dtype: object


In [65]:
BoW = CountVectorizer(
    analyzer='word',
    tokenizer=fun,
    preprocessor=fun,
    token_pattern=None)  

BoW.fit(X_train) #vær opmærksom på at vocab nu er relativt til træningssættet, men det skal det jo sådan set også være, ellers ville der være leakage
BoW.vocabulary_
X_BoW = X_train.apply(lambda x: BoW.transform([x]))


print(BoW.transform([X_train[0]]))

#tror det er rigtigt??


  (0, 36)	11
  (0, 114)	5
  (0, 115)	2
  (0, 213)	1
  (0, 283)	1
  (0, 345)	1
  (0, 650)	1
  (0, 762)	1
  (0, 861)	1
  (0, 884)	1
  (0, 1013)	1
  (0, 1088)	1
  (0, 1134)	1
  (0, 1323)	1
  (0, 1431)	2
  (0, 1446)	4
  (0, 1453)	3
  (0, 1662)	2
  (0, 1668)	1
  (0, 1739)	6
  (0, 2002)	1
  (0, 2049)	1
  (0, 2082)	4
  (0, 2120)	1
  (0, 2134)	1
  :	:
  (0, 7999)	1
  (0, 8034)	1
  (0, 8038)	1
  (0, 8077)	2
  (0, 8116)	1
  (0, 8321)	1
  (0, 8330)	1
  (0, 8481)	1
  (0, 8614)	1
  (0, 8673)	3
  (0, 8735)	1
  (0, 8832)	1
  (0, 8990)	1
  (0, 8993)	2
  (0, 9054)	1
  (0, 9302)	1
  (0, 9323)	1
  (0, 9343)	4
  (0, 9363)	1
  (0, 9515)	1
  (0, 9577)	1
  (0, 9584)	1
  (0, 9586)	4
  (0, 9602)	1
  (0, 9651)	1
